#  Inférence des CONSEQUENCES dysfonctionnement  - Stratégie Multilabels
Dans ce Notebook, nous cosntruisons un modèle qui permet d'inférer DYSFONCTIONNEMENT à partir de la classification de l'incident et des données textuelles

Nous considérons ce problème comme un problème de classification multiclasses et multilabels. En effet, il y a plusieurs effets possibles et un incidents peut entrainer plusieurs effets.

Ainsi, notre métrique d'évaluation sera le f1_samples

Dans le Notebook précedent, nous n'avions pas pris en compte l'aspect multilabel et notre score était de  f1_weighted = 0,28.

Volontairement dans un premier temps, nous ne modifions pas les paramères de notre modèle afin d'avaluer l'apport de la stratégie multilabelles.

In [38]:
import warnings
warnings.filterwarnings('ignore')

from pprint import pprint
from time import time
import logging

import pandas as pd

import numpy as np
import sklearn as sk
import seaborn as sns

import nltk
from nltk import word_tokenize
lang ='french'

import clean_text
import skmultilearn




import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit, KFold, ShuffleSplit
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score,classification_report
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD,IncrementalPCA,SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier

import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

## 0.1 Chargement des données

In [30]:
%time
df_declaration_mrv = pd.read_csv("data/data_mrv/declaration_mrv_complet.csv")#delimiter=';',encoding='ISO-8859-1')
id_to_dco = pd.read_csv("data/ref_MRV/referentiel_dispositif.csv",delimiter=';',encoding='ISO-8859-1')
df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 86.8 µs


## 0.2 Netoyage des données :
- Elimination de l'effet : PAS D'EFFET NEFASTE DECLARE
- Suppression des classes sous représentées (<15 occurences)
- Netoyyage des donénes textuelles
- Encodage de la classification et des effets

In [31]:
df_declaration_mrv.columns

Index(['Unnamed: 0', 'NUMERO_DECLARATION', 'TYPE_DECLARATION',
       'TYPE_DECLARANT', 'NB_PATIENT_CONCERNE', 'NB_DISPOSITIF_CONCERNE',
       'DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'ACTION_PATIENT', 'DCO_ID',
       'DCO', 'LIBELLE_COMMERCIAL', 'REFERENCE_COMMERCIALE', 'NUMERO_SERIE',
       'NUMERO_LOT', 'FABRICANT', 'MANDATAIRE', 'DISTRIBUTEUR', 'TDY_ID',
       'TYPE_DYSFONCTIONNEMENT', 'CDY_ID', 'CONSEQUENCE_DYSFONCTIONNEMENT',
       'TEF_ID', 'TYPE_EFFET', 'GRAVITE', 'NUMERO', 'TYPE_VIGILANCE',
       'CLASSIFICATION'],
      dtype='object')

In [68]:
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')
df_csq = df_declaration_mrv[['CDY_ID', 'CONSEQUENCE_DYSFONCTIONNEMENT']].drop_duplicates()

,CDY_ID,CONSEQUENCE_DYSFONCTIONNEMENT
0,C0,PAS DE CONSEQUENCE
12,C1032,FUITE
89,C1097,RETRAIT DU DM
113,C1098,HYSTERECTOMIE - SALPINGECTOMIE
195,C1037,MISE EN ALARME
...,...,...
70906,C1409,REMPLACEMENT PAR LE PRATICIEN
70921,C1398,AMENAGEMENT DU TRAITEMENT
72164,C1417,SECTION DE L'IMPLANT
72463,C1403,EXPLANTATION


In [32]:
%%time
#On complète les effets vide comme étant sans effets
df_declaration_mrv['TYPE_EFFET']  = df_declaration_mrv['TYPE_EFFET'].fillna("PAS D'EFFET NEFASTE DECLARE")
df_declaration_mrv['TEF_ID']= df_declaration_mrv['TEF_ID'].fillna('E1213')

#
df_declaration_mrv['TYPE_DYSFONCTIONNEMENT']  = df_declaration_mrv['TYPE_DYSFONCTIONNEMENT'].fillna("PAS DE DYSFONCTIONNEMENT")
df_declaration_mrv['TDY_ID']  = df_declaration_mrv['TDY_ID'].fillna("D0")

#
df_declaration_mrv['CONSEQUENCE_DYSFONCTIONNEMENT']  = df_declaration_mrv['CONSEQUENCE_DYSFONCTIONNEMENT'].fillna("PAS DE CONSEQUENCE")
df_declaration_mrv['CDY_ID']  = df_declaration_mrv['CDY_ID'].fillna("C0")

#on selectionne les colonnes avec des effets
df = df_declaration_mrv[['DESCRIPTION_INCIDENT','TYPE_VIGILANCE','LIBELLE_COMMERCIAL',
                         'REFERENCE_COMMERCIALE','ETAT_PATIENT','FABRICANT','DCO_ID',
                         'ACTION_PATIENT','CLASSIFICATION','CDY_ID']][df_declaration_mrv['CDY_ID']!='C0']

# On complète les NaN avec du vide
df['ETAT_PATIENT'] = df['ETAT_PATIENT'].fillna("")
df['DESCRIPTION_INCIDENT'] = df['DESCRIPTION_INCIDENT'].fillna("")
df['LIBELLE_COMMERCIAL'] = df['LIBELLE_COMMERCIAL'].fillna("")
df['FABRICANT'] = df['FABRICANT'].fillna("")
df["REFERENCE_COMMERCIALE"] = df['REFERENCE_COMMERCIALE'].fillna("")
df['TYPE_VIGILANCE'] = df['TYPE_VIGILANCE'].fillna("")
df['CLASSIFICATION'] = df['CLASSIFICATION'].fillna('')
df['DCO_ID'] = df['DCO_ID'].fillna(-1)
#On nettoieles variables textueelles : 

for col in  ['DESCRIPTION_INCIDENT','LIBELLE_COMMERCIAL','ETAT_PATIENT','FABRICANT','ACTION_PATIENT'] :
    df[col] = df[col].map(lambda x: clean_text.preprocess_text(x))


print(len(df))
n = 15
# On filtre pour a voir plus de n observations par classse
df = df.groupby("CDY_ID").filter(lambda x: len(x) > n)
print(len(df))
le_v = LabelEncoder()
df.TYPE_VIGILANCE = le_v.fit_transform(df.TYPE_VIGILANCE.values)
le = LabelEncoder()
df.CDY_ID = le.fit_transform(df.CDY_ID.values)

df_m = df.groupby('DESCRIPTION_INCIDENT')['CDY_ID'].apply(list).reset_index(name='multilabels')


df_ = pd.merge(df,df_m, on = 'DESCRIPTION_INCIDENT')
df_['multilabels'] = df_['multilabels'].apply(np.array)
df_['multilabels'] = df_['multilabels'].map(np.unique)

#df_.to_csv('Multilabel_dataset.csv')

#df_ = df_.drop_duplicates('DESCRIPTION_INCIDENT')

23452
23354
CPU times: user 9.04 s, sys: 20 ms, total: 9.06 s
Wall time: 9.06 s


In [69]:
le.inverse_transform(0)

array(['C1021', 'C1023', 'C1024', 'C1025', 'C1027', 'C1028', 'C1030',
       'C1031', 'C1032', 'C1033', 'C1035', 'C1036', 'C1037', 'C1038',
       'C1039', 'C1040', 'C1041', 'C1042', 'C1043', 'C1044', 'C1045',
       'C1046', 'C1047', 'C1048', 'C1052', 'C1078', 'C1097', 'C1098',
       'C1117', 'C1137', 'C1257', 'C1277', 'C1297', 'C1298', 'C1320',
       'C1357', 'C1402', 'C1404', 'C1405', 'C1411'], dtype=object)

## 0.3 Construction du jeu de données d'entrainement et de test

In [33]:
# selection des train et test set
train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(df_, groups=df_['DESCRIPTION_INCIDENT']))
df_train, df_test = df_.iloc[train_index], df_.iloc[test_index]
y = df_train.multilabels
y_test =df_test.multilabels

## 0.4 Encodage multilabel

In [60]:
lb = MultiLabelBinarizer()
y_lb =lb.fit_transform(y) 
y_test_lb = lb.transform(y_test)
X = df_train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
X_test = df_test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]

## 1.1 Construction du pipeline avec une stratégie ONE-VS-REST

This strategy, also known as one-vs-all, is implemented in OneVsRestClassifier. The strategy consists in fitting one classifier per class. For each classifier, the class is fitted against all the other classes. In addition to its computational efficiency (only n_classes classifiers are needed), one advantage of this approach is its interpretability. Since each class is represented by one and only one classifier, it is possible to gain knowledge about the class by inspecting its corresponding classifier. This is the most commonly used strategy and is a fair default choice.

In [61]:
%%time
preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT'),
    
    ('classification_enc', TfidfVectorizer(sublinear_tf=True, min_df=5,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 100,norm = 'l2'),'CLASSIFICATION')
     ],
    
    remainder='passthrough')


pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced'),cv=3, method='isotonic'))),
])

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 5.84 ms


## 1.2 Evaluation du pipeline en cross validation : l'importance de la séparation train/test

In [39]:
%%time
cv = KFold(n_splits=5, random_state=0)
result= cross_validate(pipeline, X, y_lb, scoring='f1_samples', cv=cv,n_jobs=-1)
result['test_score'].mean()

CPU times: user 296 ms, sys: 472 ms, total: 768 ms
Wall time: 30 s


nan

In [40]:
%%time
cv = ShuffleSplit(n_splits=5, random_state=0)
result= cross_validate(pipeline, X, y_lb, scoring='f1_samples', cv=cv,n_jobs=-1)
result['test_score'].mean()

CPU times: user 212 ms, sys: 148 ms, total: 360 ms
Wall time: 37 s


0.9019684899982296

## 1.3 Si on supprimer les doublons ?

In [42]:
## Supression des doublons
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


df_s = df_.drop_duplicates('DESCRIPTION_INCIDENT')

lb_s = MultiLabelBinarizer()
y_s = df_s.multilabels
y_s  = lb_s.fit_transform(y_s)

df_s = df_s[['FABRICANT','CLASSIFICATION','TYPE_VIGILANCE','DESCRIPTION_INCIDENT','ETAT_PATIENT']]

mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)

train_index_s,test_index_s = next(mskf.split(df_s,y_s))
df_train_s, df_test_s = df_s.iloc[train_index_s], df_s.iloc[test_index_s]
y_train_s = y_s[train_index_s]
y_test_s =y_s[test_index_s]

#lb_s = MultiLabelBinarizer()
#y_train_lb_s =lb_s.fit_transform(y_train_s) 
#y_test_lb_s = lb.transform(y_test_s)
X_train_s = df_train_s[['FABRICANT','CLASSIFICATION','TYPE_VIGILANCE','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
X_test_s = df_test_s[['FABRICANT','CLASSIFICATION','TYPE_VIGILANCE','DESCRIPTION_INCIDENT','ETAT_PATIENT']]

pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

pipeline.fit(X_train_s,y_train_s)
#y_test_lb = lb.transform(y_test)
Y_pred_ovr_s = pipeline.predict(X_test_s)
f1 = f1_score(y_test_s , Y_pred_ovr_s,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.6107124010554089


In [43]:
print(classification_report(y_test_s , Y_pred_ovr_s))

              precision    recall  f1-score   support

           0       0.57      0.58      0.57        64
           1       1.00      0.09      0.17        11
           2       1.00      0.42      0.59        36
           3       0.33      0.12      0.18         8
           4       1.00      0.08      0.15        12
           5       0.32      0.62      0.42        39
           6       0.67      0.69      0.68        93
           7       0.63      0.79      0.70       121
           8       0.61      0.56      0.58       348
           9       0.47      0.50      0.49       151
          10       0.00      0.00      0.00         5
          11       0.84      0.52      0.64        31
          12       0.74      0.66      0.70       235
          13       0.00      0.00      0.00         6
          14       1.00      0.50      0.67         2
          15       0.88      0.58      0.70        24
          16       0.44      0.27      0.33        45
          17       0.65    

## 1.3 Evaluation du pipeline sur les données de test
### Avec le SVM probabilisé

In [62]:
%%time
pipeline.fit(X,y_lb)
#y_test_lb = lb.transform(y_test)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.7317596882123686
CPU times: user 28 s, sys: 76 ms, total: 28.1 s
Wall time: 28.1 s


In [45]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.33      0.50      0.40        22
           1       0.00      0.00      0.00         1
           2       0.77      0.67      0.71        30
           3       0.00      0.00      0.00        13
           4       1.00      0.50      0.67         4
           5       0.27      0.30      0.28        27
           6       0.76      0.76      0.76        59
           7       0.62      0.77      0.69        70
           8       0.69      0.68      0.69       218
           9       0.47      0.50      0.48       125
          10       0.00      0.00      0.00         0
          11       0.89      0.77      0.83        22
          12       0.73      0.73      0.73       138
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         6
          15       0.86      0.46      0.60        13
          16       0.85      0.52      0.64        33
          17       0.75    

In [63]:
import joblib
filename = 'Encoder_consequence.sav'
joblib.dump(le, filename)
filename = 'CONSEQUENCE_model_0_73.sav'
joblib.dump(pipeline, filename)

['CONSEQUENCE_model_0_73.sav']

### Sans le SVM probabilisé

In [46]:
pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

In [47]:
%%time
pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.7387013751011102
CPU times: user 15.4 s, sys: 32 ms, total: 15.5 s
Wall time: 15.5 s


In [48]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.33      0.50      0.40        22
           1       0.00      0.00      0.00         1
           2       0.77      0.67      0.71        30
           3       0.00      0.00      0.00        13
           4       1.00      0.50      0.67         4
           5       0.27      0.30      0.28        27
           6       0.76      0.76      0.76        59
           7       0.62      0.77      0.69        70
           8       0.69      0.68      0.69       218
           9       0.47      0.50      0.48       125
          10       0.00      0.00      0.00         0
          11       0.89      0.77      0.83        22
          12       0.73      0.73      0.73       138
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         6
          15       0.86      0.46      0.60        13
          16       0.85      0.52      0.64        33
          17       0.75    

In [49]:
print('f1_score samples : ',f1)

f1_score samples :  0.7387013751011102


In [50]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.33      0.50      0.40        22
           1       0.00      0.00      0.00         1
           2       0.77      0.67      0.71        30
           3       0.00      0.00      0.00        13
           4       1.00      0.50      0.67         4
           5       0.27      0.30      0.28        27
           6       0.76      0.76      0.76        59
           7       0.62      0.77      0.69        70
           8       0.69      0.68      0.69       218
           9       0.47      0.50      0.48       125
          10       0.00      0.00      0.00         0
          11       0.89      0.77      0.83        22
          12       0.73      0.73      0.73       138
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         6
          15       0.86      0.46      0.60        13
          16       0.85      0.52      0.64        33
          17       0.75    

## Commentaire : 
#### **Les principaux enseignements sont les suivants :**
1)  L'approche multilabels est pertinente : Nous obtenons enfin un score "correct" 0.55 qui se rapproche de la précision à 4 du model simple Label. Ce qui est assez cohérent car il y a en moyenne 4 classes par evénement. Cela signifie que nous avons simplement tirer partie de la métrique mais que notre modèle n'apprend pas mieux... Dommage ! C'est en fait assez logique car le SVM en multiclasse applique déja une stratégie One vs REST

2)  Elle est pertinente.. Mais moins qu'on pourrait le penser avec la shuffle validation : comme le montre la différence entre le Kfold et le ShuffleSplit, les doublons joue un rôle important. Cette différence s'explique apr le fait que les doublons joue un rôle important dans notre base de données car ils ne sont pas exactement des "doublons". De plus ils nous permettent, de tenir compte de la proportion des classes dans notre evaluation.

3) Cette approche multilabels combinée avec la probalisation du SVM est couteuse en temps de construction de modèle et nous devrons le prendre en compte dans notre approche de finetuning.

#### **Les pistes d'améliorations sont :**

1) Finetuner le modèle pour vérifier que notre jeu de paramètre n'est pas un cas particulier de performances 


2) Essayer une approche
- MultiOutputClassifier
- ClassifierChain https://scikit-learn.org/stable/auto_examples/multioutput/plot_classifier_chain_yeast.html

3) Essayer le one-sht learning

## 2.1 Optune et la marge de progression du modèle
Pour le finetunning, nous enlevons la probabilisation du modèle

In [180]:
pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

In [184]:
import optuna
from optuna import Trial

def objective(trial):    
    
    train_index,test_index = next(GroupShuffleSplit(random_state=1029).split(X, groups=X['DESCRIPTION_INCIDENT']))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    y_train_lb = lb.transform(y_train)
    y_test_lb = lb.transform(y_test)
    
    #Etat Patient
    #vect__etat_pat_tfidf__analyzer = trial.suggest_categorical('vect__etat_pat_tfidf__analyzer', ['word', 'char', 'char_wb']) 
    vect__etat_pat_tfidf__max_features = trial.suggest_int('vect__etat_pat_tfidf__max_features', 500, 20_000)
    vect__etat_pat_tfidf__min_df =  trial.suggest_int('vect__etat_pat_tfidf__min_df', 1,5)
    vect__etat_pat_tfidf__norm = trial.suggest_categorical('vect__etat_pat_tfidf__norm', ('l1', 'l2'))
    #Description
    #vect__description_tfidf__analyzer = trial.suggest_categorical('vect__description_tfidf__analyzer', ['word', 'char', 'char_wb']) 
    vect__description_tfidf__max_features = trial.suggest_int('vect__description_tfidf__max_features', 1500, 60_000)
    vect__description_tfidf__min_df =  trial.suggest_int('vect__description_tfidf__min_df', 1,5)
    vect__description_tfidf__norm = trial.suggest_categorical('vect__description_tfidf__norm', ('l1', 'l2'))
    #Fabricant
    vect__fabricant_tfidf__analyzer = trial.suggest_categorical('vect__fabricant_tfidf__analyzer', ['word', 'char', 'char_wb']) 
    vect__fabricant_tfidf__max_features = trial.suggest_int('vect__fabricant_tfidf__max_features', 500, 10_000)
    vect__fabricant_tfidf__min_df =  trial.suggest_int('vect__fabricant_tfidf__min_df', 1,5)
    vect__fabricant_tfidf__norm = trial.suggest_categorical('vect__fabricant_tfidf__norm', ('l1', 'l2'))
    #action patient
    
    #Classification
    vect__classification_enc__analyzer = trial.suggest_categorical('vect__classification_enc__analyzer', ['word', 'char', 'char_wb']) 
    vect__classification_enc__max_features = trial.suggest_int('vect__classification_enc__max_features', 500, 5000)
    vect__classification_enc__min_df =  trial.suggest_int('vect__classification_enc__min_df', 1,5)
    vect__classification_enc__norm = trial.suggest_categorical('vect__classification_enc__norm', ('l1', 'l2'))
    
    #clf__C =trial.suggest_loguniform('svr_c', 1e-5, 1e5)
    

    
    params = {
        #'vect__etat_pat_tfidf__analyzer':vect__etat_pat_tfidf__analyzer,
        'vect__etat_pat_tfidf__max_features': vect__etat_pat_tfidf__max_features,
        'vect__etat_pat_tfidf__min_df':vect__etat_pat_tfidf__min_df,
        'vect__etat_pat_tfidf__norm':vect__etat_pat_tfidf__norm,
        
        #'vect__description_tfidf__analyzer':vect__description_tfidf__analyzer,
        'vect__description_tfidf__max_features': vect__description_tfidf__max_features,
        'vect__description_tfidf__min_df':vect__description_tfidf__min_df,
        'vect__description_tfidf__norm':vect__description_tfidf__norm,
        
        'vect__fabricant_tfidf__analyzer':vect__fabricant_tfidf__analyzer,
        'vect__fabricant_tfidf__max_features': vect__fabricant_tfidf__max_features,
        'vect__fabricant_tfidf__min_df':vect__fabricant_tfidf__min_df,
        'vect__fabricant_tfidf__norm':vect__fabricant_tfidf__norm,
        
        
        'vect__classification_enc__analyzer':vect__classification_enc__analyzer,
        'vect__classification_enc__max_features': vect__classification_enc__max_features,
        'vect__classification_enc__min_df':vect__classification_enc__min_df,
        'vect__classification_enc__norm':vect__classification_enc__norm,
        
        #'clf__C':clf__C
    }
    
    pipeline.set_params(**params)
    pipeline.fit(X_train,y_train_lb)
    Y_pred_ovr = pipeline.predict(X_test)
    score = f1_score(y_test_lb , Y_pred_ovr,average='samples')
    
    return score

In [185]:
time_out = 75*100
studyName = 'EFFET_ml_optimisation_svm'
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=time_out)

#Suvegarde du resultat
df = study.trials_dataframe()
df.to_json(studyName+'.json')
print(study.best_trial)

[I 2020-06-10 18:43:30,547] Finished trial#0 with value: 0.5268400357946823 with parameters: {'vect__etat_pat_tfidf__max_features': 9211, 'vect__etat_pat_tfidf__min_df': 4, 'vect__etat_pat_tfidf__norm': 'l2', 'vect__description_tfidf__max_features': 34910, 'vect__description_tfidf__min_df': 4, 'vect__description_tfidf__norm': 'l1', 'vect__fabricant_tfidf__analyzer': 'char_wb', 'vect__fabricant_tfidf__max_features': 7623, 'vect__fabricant_tfidf__min_df': 3, 'vect__fabricant_tfidf__norm': 'l1', 'vect__action_pat_tfidf__analyzer': 'char', 'vect__classification_enc__max_features': 3078, 'vect__classification_enc__min_df': 5, 'vect__classification_enc__norm': 'l1'}. Best is trial#0 with value: 0.5268400357946823.
[I 2020-06-10 18:45:45,297] Finished trial#1 with value: 0.5314562900155817 with parameters: {'vect__etat_pat_tfidf__max_features': 2892, 'vect__etat_pat_tfidf__min_df': 1, 'vect__etat_pat_tfidf__norm': 'l2', 'vect__description_tfidf__max_features': 36279, 'vect__description_tfidf_

FrozenTrial(number=13, value=0.5638014529635027, datetime_start=datetime.datetime(2020, 6, 10, 19, 7, 53, 904403), datetime_complete=datetime.datetime(2020, 6, 10, 19, 9, 11, 38205), params={'vect__etat_pat_tfidf__max_features': 19700, 'vect__etat_pat_tfidf__min_df': 4, 'vect__etat_pat_tfidf__norm': 'l2', 'vect__description_tfidf__max_features': 20401, 'vect__description_tfidf__min_df': 2, 'vect__description_tfidf__norm': 'l2', 'vect__fabricant_tfidf__analyzer': 'word', 'vect__fabricant_tfidf__max_features': 9992, 'vect__fabricant_tfidf__min_df': 3, 'vect__fabricant_tfidf__norm': 'l1', 'vect__action_pat_tfidf__analyzer': 'word', 'vect__classification_enc__max_features': 2062, 'vect__classification_enc__min_df': 4, 'vect__classification_enc__norm': 'l1'}, distributions={'vect__etat_pat_tfidf__max_features': IntUniformDistribution(high=20000, low=500, step=1), 'vect__etat_pat_tfidf__min_df': IntUniformDistribution(high=5, low=1, step=1), 'vect__etat_pat_tfidf__norm': CategoricalDistribut

In [51]:
%%time
Param = {'vect__etat_pat_tfidf__max_features': 19700, 'vect__etat_pat_tfidf__min_df': 4, 'vect__etat_pat_tfidf__norm': 'l2', 
         'vect__description_tfidf__max_features': 20401, 'vect__description_tfidf__min_df': 2, 'vect__description_tfidf__norm': 'l2', 
         'vect__fabricant_tfidf__analyzer': 'word', 'vect__fabricant_tfidf__max_features': 9992, 'vect__fabricant_tfidf__min_df': 3, 
         'vect__fabricant_tfidf__norm': 'l1', 'vect__classification_enc_analyzer': 'word', 'vect__classification_enc__max_features': 2062,
         'vect__classification_enc__min_df': 4, 'vect__classification_enc__norm': 'l1'}

preprocess2 = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=2,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 20401,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=4,
                                        ngram_range=(1, 1),
                                        stop_words=STOP_WORDS,
                                        max_features = 19700,
                                        norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 9992,norm = 'l1'), 'FABRICANT'),
    
    ('classification_enc', TfidfVectorizer(sublinear_tf=True, min_df=4,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 2062,norm = 'l2'),'CLASSIFICATION')
     ],
    
    remainder='passthrough')


pipeline = Pipeline([
    ('vect', preprocess2),
    ('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced'),cv=3, method='isotonic'))),
])


pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.7410986101919259
CPU times: user 30.4 s, sys: 52 ms, total: 30.5 s
Wall time: 30.5 s


In [52]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.26      0.32      0.29        22
           1       0.00      0.00      0.00         1
           2       0.95      0.60      0.73        30
           3       0.00      0.00      0.00        13
           4       1.00      0.50      0.67         4
           5       0.00      0.00      0.00        27
           6       0.84      0.54      0.66        59
           7       0.72      0.66      0.69        70
           8       0.76      0.54      0.63       218
           9       0.72      0.26      0.39       125
          10       0.00      0.00      0.00         0
          11       0.81      0.77      0.79        22
          12       0.81      0.59      0.69       138
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         6
          15       0.86      0.46      0.60        13
          16       0.86      0.18      0.30        33
          17       0.82    

## Commantaire
 Notre première approximation des performances était la bonne. En effet, le finetunning des paramètres des TFIDF n'a pas permis d'augmenter significativmement les résultats
 
## 3.0 L'approche Multioutput

> Multioutput classification support can be added to any classifier with MultiOutputClassifier. This strategy consists of fitting one classifier per target. This allows multiple target variable classifications. The purpose of this class is to extend estimators to be able to estimate a series of target functions (f1,f2,f3…,fn) that are trained on a single X predictor matrix to predict a series of responses (y1,y2,y3…,yn).

https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html




In [53]:
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', MultiOutputClassifier(LinearSVC(class_weight='balanced'))),
])
#### prédiction 
pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.7387013751011102


In [54]:
print(classification_report(y_test_lb , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.33      0.50      0.40        22
           1       0.00      0.00      0.00         1
           2       0.77      0.67      0.71        30
           3       0.00      0.00      0.00        13
           4       1.00      0.50      0.67         4
           5       0.27      0.30      0.28        27
           6       0.76      0.76      0.76        59
           7       0.62      0.77      0.69        70
           8       0.69      0.68      0.69       218
           9       0.47      0.50      0.48       125
          10       0.00      0.00      0.00         0
          11       0.89      0.77      0.83        22
          12       0.73      0.73      0.73       138
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         6
          15       0.86      0.46      0.60        13
          16       0.85      0.52      0.64        33
          17       0.75    

### Commentaire
Comme attendu, nous n'observons pas de grande différence car les deux approches sont très similaires

## 3.1 Approche One vs One

>This strategy consists in fitting one classifier per class pair. At prediction time, the class which received the most votes is selected. Since it requires to fit n_classes * (n_classes - 1) / 2 classifiers, this method is usually slower than one-vs-the-rest, due to its O(n_classes^2) complexity. However, this method may be advantageous for algorithms such as kernel algorithms which don’t scale well with n_samples. This is because each individual learning problem only involves a small subset of the data whereas, with one-vs-the-rest, the complete dataset is used n_classes times.



In [55]:
%%time
from sklearn.multiclass import OneVsOneClassifier
pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', MultiOutputClassifier(OneVsOneClassifier(LinearSVC(class_weight='balanced')))),
])
#### prédiction 
pipeline.fit(X,y_lb)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test_lb , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.7387013751011102
CPU times: user 17.1 s, sys: 32 ms, total: 17.1 s
Wall time: 17.1 s


### Commentaire
Nous n'oservons pas de changement de performances, seulement une hausse du temps de calcul
## 3.2 l'approche ClassifierChain
>A multi-label model that arranges binary classifiers into a chain.
Each model makes a prediction in the order specified by the chain using all of the available features provided to the model plus the predictions of models that are earlier in the chain.



In [56]:
from sklearn.multioutput import ClassifierChain

In [57]:
train_index_,test_index_ = next(GroupShuffleSplit(random_state=1029).split(X, groups=X['DESCRIPTION_INCIDENT']))
X_train_, X_test_ = X.iloc[train_index_], X.iloc[test_index_]
y_train_, y_test_ = y.iloc[train_index_], y.iloc[test_index_]

X_train_, X_test_ =preprocess.fit_transform(X_train_),preprocess.transform(X_test_)
y_train_lb_ = lb.transform(y_train_)
y_test_lb_ = lb.transform(y_test_)

In [58]:
%%time
X_train, X_test_ =preprocess.fit_transform(X),preprocess.transform(X_test)
clf = LinearSVC(class_weight='balanced')


chains = [ClassifierChain(clf, order='random', random_state=i) for i in range(10)]

for chain in chains:
    chain.fit(X_train, y_lb)
    
y_pred_chains = np.array([chain.predict(X_test_) for chain in chains])

chain_f1_scores = [f1_score(y_test_lb, y_pred_chain, average='samples') for y_pred_chain in y_pred_chains]

y_pred_ensemble = y_pred_chains.mean(axis=0)

y_e = y_pred_ensemble>=0.4

ensemble_f1_score = f1_score(y_test_lb,y_e, average='samples')

print(ensemble_f1_score)

0.7790352231781749
CPU times: user 2min 8s, sys: 492 ms, total: 2min 8s
Wall time: 2min 8s


In [59]:
print(classification_report(y_test_lb,y_e))

              precision    recall  f1-score   support

           0       0.33      0.50      0.40        22
           1       0.00      0.00      0.00         1
           2       0.77      0.67      0.71        30
           3       0.00      0.00      0.00        13
           4       1.00      0.50      0.67         4
           5       0.27      0.30      0.28        27
           6       0.71      0.85      0.78        59
           7       0.62      0.86      0.72        70
           8       0.58      0.75      0.66       218
           9       0.49      0.59      0.53       125
          10       0.00      0.00      0.00         0
          11       0.86      0.86      0.86        22
          12       0.67      0.72      0.69       138
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         6
          15       0.67      0.46      0.55        13
          16       0.86      0.55      0.67        33
          17       0.76    

In [224]:
#Random test
test = np.random.rand(y_e.shape[0],y_e.shape[1])>=0.5
f1_score(y_test_lb_,test, average='samples')

0.04225001471861698

### Commentaire : 
L'approche chain combiner à une méthode ensemblise améliore les performaces de quelques pourcent mais l'ordre de grandeur reste le même.

L'approche multilabel est une piste à suivre mais elle n'améliore pas ou peu la qualité de notre apprentissage.

In [247]:
C = sk.metrics.multilabel_confusion_matrix(y_test_lb,y_e)

In [257]:
df_re = pd.DataFrame(classification_report(y_test_lb , Y_pred_ovr,output_dict=True)).T

In [273]:
df_re['Class'] = le.inverse_transform(df_re.index.map(int).values[:-5])

ValueError: invalid literal for int() with base 10: 'micro avg'

In [269]:
L =df_re.index.to_list()

In [271]:
L.map(int)

AttributeError: 'list' object has no attribute 'map'